# Activity 1 – Data Visualization and Dashboard Deployment

This notebook follows the steps requested in the assignment: exploratory data analysis, basic visualizations, and preparation of a Streamlit dashboard.

## 1. Setup
Run this cell to install any missing packages

In [24]:
# If you are in Google Colab, you can uncomment these lines if needed
# !pip install pandas matplotlib seaborn streamlit plotly
print('Setup cell executed.')

Setup cell executed.


## 2. Import libraries

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path

sns.set(style='whitegrid')
print('Libraries imported')

Libraries imported


## 3. Load dataset
In Colab, upload the file `university_student_data.csv` in the left panel (Files) and make sure the path below matches.

In [4]:
from google.colab import files
uploaded = files.upload()

Saving university_student_data.csv to university_student_data.csv


In [ ]:
data_path = Path('university_student_data.csv')
if not data_path.exists():
    raise FileNotFoundError('Please upload university_student_data.csv to the current working directory.')

df = pd.read_csv(data_path)
print('Dataset loaded. Shape:', df.shape)
df.head()

## 4. Inspect columns
Here we display basic information to understand which columns we have and what they mean.

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

### Column documentation

- `year`: Academic year of the data record.
- `term`: Academic term (e.g., **Spring** or **Fall**).
- `applications`: Number of students who applied for admission.
- `admitted`: Number of students who were accepted to the university.
- `enrolled`: Number of students who actually enrolled after admission.
- `retention_rate`: Percentage of students retained in the following academic period.
- `satisfaction_score`: Average student satisfaction percentage.
- `engineering_enrolled`: Number of enrolled students in the **Engineering** department.
- `business_enrolled`: Number of enrolled students in the **Business** department.
- `arts_enrolled`: Number of enrolled students in the **Arts** department.
- `science_enrolled`: Number of enrolled students in the **Science** department.

## 5. Visualizations required in the assignment

In [ ]:
# Clean and rename columns to standard format
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('(', '')
    .str.replace(')', '')
    .str.replace('%', '')
)

# Rename the key columns to match the code expectations
df = df.rename(columns={
    'student_satisfaction_': 'satisfaction_score',
    'retention_rate_': 'retention_rate'
})

print("Cleaned columns:", df.columns.tolist())

In [ ]:
# 5.1 Retention rate trends over time
if 'year' in df.columns and 'retention_rate' in df.columns:
    retention_by_year = df.groupby('year')['retention_rate'].mean().reset_index()
    plt.figure(figsize=(8,4))
    sns.lineplot(data=retention_by_year, x='year', y='retention_rate', marker='o')
    plt.title('Retention Rate Over Time')
    plt.xlabel('Year')
    plt.ylabel('Retention Rate (%)')
    plt.tight_layout()
    plt.show()
else:
    print('Columns year/retention_rate not found in the dataset.')

In [ ]:
# 5.2 Student satisfaction scores by year
if 'year' in df.columns and 'satisfaction_score' in df.columns:
    sat_by_year = df.groupby('year')['satisfaction_score'].mean().reset_index()
    plt.figure(figsize=(8,4))
    sns.barplot(data=sat_by_year, x='year', y='satisfaction_score')
    plt.title('Average Student Satisfaction by Year')
    plt.xlabel('Year')
    plt.ylabel('Satisfaction Score')
    plt.tight_layout()
    plt.show()
else:
    print('Columns year/satisfaction_score not found in the dataset.')

In [ ]:
# 5.3 Comparison between Spring and Fall terms
if 'term' in df.columns and 'retention_rate' in df.columns:
    term_comp = df.groupby('term')['retention_rate'].mean().reset_index()
    plt.figure(figsize=(6,4))
    sns.barplot(data=term_comp, x='term', y='retention_rate')
    plt.title('Retention Rate by Term')
    plt.xlabel('Term')
    plt.ylabel('Retention Rate (%)')
    plt.tight_layout()
    plt.show()
else:
    print('Columns term/retention_rate not found in the dataset.')

## 6. Streamlit dashboard prototype
The following cell creates a basic `app.py` file that you can push to GitHub and deploy on Streamlit Cloud.

In [33]:
%%writefile app.py
import pandas as pd
import streamlit as st
import plotly.express as px

st.set_page_config(page_title='University Dashboard', layout='wide')

@st.cache_data
def load_data(path='university_student_data.csv'):
    return pd.read_csv(path)

df = load_data()

st.title('University Student Analytics Dashboard')
st.markdown('This dashboard summarizes admissions, enrollment, retention, and satisfaction data.')

# Sidebar filters
years = sorted(df['year'].dropna().unique()) if 'year' in df.columns else []
departments = sorted(df['department'].dropna().unique()) if 'department' in df.columns else []
terms = sorted(df['term'].dropna().unique()) if 'term' in df.columns else []

year_filter = st.sidebar.multiselect('Select year(s):', years, default=years)
dept_filter = st.sidebar.multiselect('Select department(s):', departments, default=departments)
term_filter = st.sidebar.multiselect('Select term(s):', terms, default=terms)

df_f = df.copy()
if year_filter:
    df_f = df_f[df_f['year'].isin(year_filter)]
if dept_filter:
    df_f = df_f[df_f['department'].isin(dept_filter)]
if term_filter:
    df_f = df_f[df_f['term'].isin(term_filter)]

# KPI section
col1, col2, col3, col4 = st.columns(4)
with col1:
    st.metric('Total Applications', int(df_f['applications'].sum()) if 'applications' in df_f.columns else 'N/A')
with col2:
    st.metric('Total Enrolled', int(df_f['enrolled'].sum()) if 'enrolled' in df_f.columns else 'N/A')
with col3:
    st.metric('Avg Retention Rate', round(df_f['retention_rate'].mean(), 2) if 'retention_rate' in df_f.columns else 'N/A')
with col4:
    st.metric('Avg Satisfaction', round(df_f['satisfaction_score'].mean(), 2) if 'satisfaction_score' in df_f.columns else 'N/A')

st.divider()

# Visualizations
if 'year' in df_f.columns and 'retention_rate' in df_f.columns:
    ret_fig = px.line(df_f.groupby('year')['retention_rate'].mean().reset_index(),
                      x='year', y='retention_rate', title='Retention Rate Over Time')
    st.plotly_chart(ret_fig, use_container_width=True)

col_a, col_b = st.columns(2)
with col_a:
    if 'year' in df_f.columns and 'satisfaction_score' in df_f.columns:
        sat_fig = px.bar(df_f.groupby('year')['satisfaction_score'].mean().reset_index(),
                         x='year', y='satisfaction_score', title='Avg Satisfaction by Year')
        st.plotly_chart(sat_fig, use_container_width=True)
with col_b:
    if 'term' in df_f.columns and 'retention_rate' in df_f.columns:
        term_fig = px.bar(df_f.groupby('term')['retention_rate'].mean().reset_index(),
                          x='term', y='retention_rate', title='Retention Rate by Term')
        st.plotly_chart(term_fig, use_container_width=True)

st.markdown('---')
st.markdown('Data preview:')
st.dataframe(df_f.head())


Overwriting app.py
